In [1]:
from Phidget22.Phidget import *
from Phidget22.Devices.DigitalInput import *
from Phidget22.Devices.Stepper import *
import time
import numpy as np

In [2]:
digitalInput0 = DigitalInput()
digitalInput1 = DigitalInput()
digitalInput2 = DigitalInput()
digitalInput3 = DigitalInput()
digitalInput4 = DigitalInput()
digitalInput5 = DigitalInput()
digitalInput6 = DigitalInput()
digitalInput7 = DigitalInput()

digitalInput0.setChannel(0)
digitalInput1.setChannel(1)
digitalInput2.setChannel(2)
digitalInput3.setChannel(3)
digitalInput4.setChannel(4)
digitalInput5.setChannel(5)
digitalInput6.setChannel(6)
digitalInput7.setChannel(7)

digitalInput0.openWaitForAttachment(5000)
digitalInput1.openWaitForAttachment(5000)
digitalInput2.openWaitForAttachment(5000)
digitalInput3.openWaitForAttachment(5000)
digitalInput4.openWaitForAttachment(5000)
digitalInput5.openWaitForAttachment(5000)
digitalInput6.openWaitForAttachment(5000)
digitalInput7.openWaitForAttachment(5000)

PhidgetException: PhidgetException 0x03 (Timed Out)
No matching devices were found to open. Make sure your device is attached, and that your addressing parameters are specified correctly. If your Phidget has a plug or terminal block for external power, ensure it is plugged in and powered.

In [ ]:
x = 0
while x < 30:
    state0 = digitalInput0.getState()
    state1 = digitalInput1.getState()
    state2 = digitalInput2.getState()
    state3 = digitalInput3.getState()
    state4 = digitalInput4.getState()
    state5 = digitalInput5.getState()
    state6 = digitalInput6.getState()
    state7 = digitalInput7.getState()
    print('----------------')
    print("State0: " + str(state0))
    print("State1: " + str(state1))
    print("State2: " + str(state2))
    print("State3: " + str(state3))
    print("State4: " + str(state4))
    print("State5: " + str(state5))
    print("State6: " + str(state6))
    print("State7: " + str(state7))
    x += 1
    time.sleep(1)

In [ ]:
digitalInput0.close()
digitalInput1.close()
digitalInput2.close()
digitalInput3.close()
digitalInput4.close()
digitalInput5.close()
digitalInput6.close()
digitalInput7.close()

Stepper Setup

In [3]:

from Phidget22.Phidget import *
from Phidget22.Devices.DigitalInput import *
from Phidget22.Devices.Stepper import *
import time
import numpy as np

mandrel = Stepper()
mandrel.setDeviceSerialNumber(615813) 
mandrel.openWaitForAttachment(5000)
mandrel.setAcceleration(80000)
mandrel.setRescaleFactor(1/60.444) #convert so set target is in degrees


carriage = Stepper()
carriage.setDeviceSerialNumber(615519) 
carriage.openWaitForAttachment(5000)
carriage.setAcceleration(50000)
carriage.setRescaleFactor(-1/3624)

head = Stepper()
head.setDeviceSerialNumber(616004) 
head.openWaitForAttachment(5000)
head.setAcceleration(4800)
head.setRescaleFactor(9/1040)

Enter parameters for hoop winding


In [2]:
linear_velocity = .2 # inch/sec 
fiber_thickness = .137
travel_distance = 16

t = fiber_thickness/linear_velocity
angular_velocity = 360/t


In [ ]:
carriage.setVelocityLimit(linear_velocity)
mandrel.setVelocityLimit(angular_velocity)

carriage.setTargetPosition(travel_distance)
mandrel.setTargetPosition(-travel_distance*(360/fiber_thickness))

In [11]:
mandrel.setEngaged(True)
carriage.setEngaged(True)
head.setEngaged(True)

# CODE STARTS HERE

## Initilization

In [1]:
from Phidget22.Phidget import *
from Phidget22.Devices.DigitalInput import *
from Phidget22.Devices.Stepper import *
import time
import numpy as np

mandrel = Stepper()
mandrel.setDeviceSerialNumber(615813) 
mandrel.openWaitForAttachment(5000)
mandrel.setAcceleration(80000)
mandrel.setRescaleFactor(1/60.444) #convert so set target is in degrees


carriage = Stepper()
carriage.setDeviceSerialNumber(615519) 
carriage.openWaitForAttachment(5000)
carriage.setAcceleration(50000)
carriage.setRescaleFactor(-1/3624)

head = Stepper()
head.setDeviceSerialNumber(616004) 
head.openWaitForAttachment(5000)
head.setAcceleration(4800)
head.setRescaleFactor(9/1040)

## calc velocities and angular distance for helical winding and hoopwinding 
enter user parameters

In [2]:
#hoop: 
linear_velocity = .2 # inch/sec 
fiber_thickness = .152
travel_distance = 15.5

t = fiber_thickness/linear_velocity
angular_velocity = 360/t

#helical 
def calc_alpha(N, B, D):
    return np.degrees(np.arccos((N*B)/(np.pi*D)))

#user parameters
alpha_desired = 55 #degrees 
# r = 1.985/2 #radius inches 
r = .5/2 #radius inches 
distance = 15.5
L = 1.6 #in/s 
fiber_thickness = .16
offset_fudge_factor = 1# percent multiplier for offset angle 
head_angle_length = .25 #in length that the head turns during 

B = fiber_thickness
D = 2*r
possible_alphas = []
alpha_error = 0
previous_alpha_error = 100
N = 5
while alpha_error< previous_alpha_error:
    N += 1
    alpha_error = abs(calc_alpha(N, B, D) - alpha_desired )
    previous_alpha_error = abs(calc_alpha(N-1, B, D) - alpha_desired )
    

alpha = calc_alpha(N-1, B, D)
print(alpha)
print(N)

width1 = fiber_thickness/np.cos(np.radians(alpha))
angular_offset = width1*(360/(2*np.pi*r))*offset_fudge_factor 

W_mand = L*np.tan(np.radians(alpha))*(360/(2*np.pi*r))
W_head = (90-alpha)/(360/W_mand)*1.5

T = distance/L 
Total_angle = W_mand*T
head_F = -(90-alpha+15)
head_B = 90-alpha+15

52.32693742226565
7


## Helical Forward?

In [3]:
carriage.setVelocityLimit(L)
mandrel.setVelocityLimit(W_mand)
print(N)
for i in range(N):
    
    print(i)
     
    #Helical Path forwards
    head.setTargetPosition(head_F)
    head.setEngaged(True)

    carriage.setTargetPosition(distance)
    mandrel.setTargetPosition(mandrel.getTargetPosition()-Total_angle)

    head.setEngaged(True)
    mandrel.setEngaged(True)
    carriage.setEngaged(True)

    while (mandrel.getIsMoving()):
        time.sleep(.5)

      
    #taper 


    head.setVelocityLimit(W_head)
    head.setTargetPosition(0)
    mandrel.setTargetPosition(mandrel.getTargetPosition() - 360)

    mandrel.setEngaged(True)
    head.setEngaged(True)

    while (mandrel.getIsMoving()):
        time.sleep(.5)

         
    # move head to helical angle
    head.setVelocityLimit(abs(head_B)/(head_angle_length/L))
    head.setTargetPosition(head_B)

    
    #return on helical path   
    carriage.setTargetPosition(0)
    mandrel.setTargetPosition(mandrel.getTargetPosition()-Total_angle)

    head.setEngaged(True)
    mandrel.setEngaged(True)
    carriage.setEngaged(True)

    while (mandrel.getIsMoving()):
        time.sleep(.5)

     
    #turn head while tapering 
    head.setVelocityLimit(W_head)
    head.setTargetPosition(0)
    mandrel.setTargetPosition(mandrel.getTargetPosition() - 360-angular_offset)

    mandrel.setEngaged(True)
    head.setEngaged(True)

    while (mandrel.getIsMoving()):
        time.sleep(.5)
        
       
    # move head to helical angle
    head.setVelocityLimit(abs(head_B)/(head_angle_length/L))
    head.setTargetPosition(head_F)


Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helic

## Hoop winding forwards: 

In [3]:
carriage.setVelocityLimit(linear_velocity)
mandrel.setVelocityLimit(angular_velocity)

head.setTargetPosition(0)
carriage.setTargetPosition(travel_distance)
mandrel.setTargetPosition(mandrel.getTargetPosition()-travel_distance*(360/fiber_thickness))

head.setEngaged(True)
mandrel.setEngaged(True)
carriage.setEngaged(True)


## Helical Winding Backwards 

In [4]:
carriage.setVelocityLimit(L)
mandrel.setVelocityLimit(W_mand)
print("Total Iterations:", N)
for i in range(N):
    
    temp_string = 'Iter:' + 
    print(temp_string)
        
    # move head to helical angle
    head.setVelocityLimit(abs(head_B)/(head_angle_length/L))
    head.setTargetPosition(head_B)

     
    #return on helical path   
    carriage.setTargetPosition(0)
    mandrel.setTargetPosition(mandrel.getTargetPosition()-Total_angle)

    head.setEngaged(True)
    mandrel.setEngaged(True)
    carriage.setEngaged(True)

    while (mandrel.getIsMoving()):
        time.sleep(.5)

     
    #turn head while tapering 
    head.setVelocityLimit(W_head)
    head.setTargetPosition(0)
    mandrel.setTargetPosition(mandrel.getTargetPosition() - 360-angular_offset)

    mandrel.setEngaged(True)
    head.setEngaged(True)

    while (mandrel.getIsMoving()):
        time.sleep(.5)
        
         
    # move head to helical angle
    head.setVelocityLimit(abs(head_B)/(head_angle_length/L))
    head.setTargetPosition(head_F)
    
      
    #Helical Path forwards
    head.setTargetPosition(head_F)
    head.setEngaged(True)

    carriage.setTargetPosition(distance)
    mandrel.setTargetPosition(mandrel.getTargetPosition()-Total_angle)

    head.setEngaged(True)
    mandrel.setEngaged(True)
    carriage.setEngaged(True)

    while (mandrel.getIsMoving()):
        time.sleep(.5)

       
    #taper 


    head.setVelocityLimit(W_head)
    head.setTargetPosition(0)
    mandrel.setTargetPosition(mandrel.getTargetPosition() - 360)

    mandrel.setEngaged(True)
    head.setEngaged(True)

    while (mandrel.getIsMoving()):
        time.sleep(.5)


moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper
moving head helical_b
Helical Path backward
Taper
moving head helical_f
Helical Forwards
Taper


## hoop winding backward

In [5]:
carriage.setVelocityLimit(linear_velocity)
mandrel.setVelocityLimit(angular_velocity)

head.setTargetPosition(0)
carriage.setTargetPosition(0)
mandrel.setTargetPosition(mandrel.getTargetPosition()-travel_distance*(360/fiber_thickness))

head.setEngaged(True)
mandrel.setEngaged(True)
carriage.setEngaged(True)

In [21]:
mandrel.setTargetPosition(200000)

In [4]:

carriage.setTargetPosition(0)
head.setTargetPosition(0)

In [22]:
val = 1
mandrel.setEngaged(val)
head.setEngaged(val)
mandrel.setEngaged(val)

In [13]:
head.close()
mandrel.close()
carriage.close()